In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Базовый датафрейм

In [2]:
target_events = ["P1 <= 0",
                "P2 <= 0",
                "T1 > max",
                "T1 < min",
                "Авария",
                "Недостаточная температура подачи в центральном отоплении (Недотоп)",
                "Превышение температуры подачи в центральном отоплении (Перетоп)",
                "Утечка теплоносителя",
                "Температура в квартире ниже нормативной",
                "Отсутствие отопления в доме",
                "Крупные пожары",
                "Температура в помещении общего пользования ниже нормативной",
                "Аварийная протечка труб в подъезде",
                "Протечка труб в подъезде",
                "Течь в системе отопления",
                "Сильная течь в системе отопления"]

In [3]:
df_autumn = pd.read_csv("../data/events_autumn/events.csv", delimiter=';')
df_spring = pd.read_csv("../data/events_spring/events.csv", delimiter=';')
# rename columns to easily access them
df_autumn.columns = ["event", "source", "date_registry", "date_close", "district", "unom", "address", "datetime_finish_event"]
df_spring.columns = ["event", "source", "date_registry", "date_close", "district", "unom", "address", "datetime_finish_event"]
events_df = pd.concat([df_autumn, df_spring])
events_df["date_registry"] = pd.to_datetime(events_df["date_registry"]).dt.date
events_df["date_close"] = pd.to_datetime(events_df["date_close"]).dt.date
events_df = events_df[events_df.event.isin(target_events)].drop(columns=["district", "address", "source", "datetime_finish_event"])
events_df = events_df[["unom", "event", "date_registry", "date_close"]]

In [4]:
building_info_df = pd.read_excel("../raw/9. Выгрузка БТИ.xlsx", skiprows=1)
building_info_df = building_info_df[["Unnamed: 11", "Материал", "Назначение", "Класс"]]
building_info_df.columns = ["unom", "material", "purpose", "class"]

In [24]:
districts_df = pd.read_csv("../data/11_chauffage.csv")
# df11 = df11[["UNOM", "Район", "Месяц/Год", "Объём поданого теплоносителя в систему ЦО", "Объём обратного теплоносителя из системы ЦО",
#       "Разница между подачей и обраткой(Подмес)", "Разница между подачей и обраткой(Утечка)", "Температура подачи",
#       "Температура обратки", "Наработка часов счётчика", "Расход тепловой энергии ", "Ошибки"]]
districts_df = districts_df[["UNOM", "Район"]]
districts_df = districts_df[["UNOM", "Район"]].drop_duplicates()
districts_df.columns = ["unom", "district"]

In [6]:
df = pd.merge(events_df, building_info_df, how="left")
df = df.merge(districts_df, how="left")

# Функции

In [7]:
def get_event_probabilities_by_district(df, district):
    filter_df = df[df["district"] == district]
    
    event_distribution = np.round(filter_df.event.value_counts() / filter_df.event.value_counts().sum() * 100, 2)
    
    return event_distribution

def get_event_probabilities(df):
    result_df = df.event.value_counts() / df.event.value_counts().sum()
    
    result_df = np.round(result_df * 100, 2)
    
    return result_df

In [8]:
get_event_probabilities(df)

event
Температура в квартире ниже нормативной                        88.23
T1 > max                                                        4.62
Отсутствие отопления в доме                                     2.12
P1 <= 0                                                         1.32
T1 < min                                                        1.26
Сильная течь в системе отопления                                0.61
Температура в помещении общего пользования ниже нормативной     0.59
Течь в системе отопления                                        0.53
P2 <= 0                                                         0.44
Протечка труб в подъезде                                        0.27
Аварийная протечка труб в подъезде                              0.00
Крупные пожары                                                  0.00
Name: count, dtype: float64

In [9]:
df.district.unique()

array(['Преображенское', 'Измайлово', nan, 'Богородское', 'Новогиреево',
       'Перово', 'Метрогородок', 'Восточное Измайлово', 'Ивановское',
       'Вешняки', 'Гольяново', 'Северное Измайлово', 'Соколиная Гора',
       'Сокольники', 'Косино-Ухтомский', 'Новокосино', 'Восточный'],
      dtype=object)

In [10]:
get_event_probabilities_by_district(df, "Ивановское")

event
Температура в квартире ниже нормативной                        89.82
T1 > max                                                        2.40
Отсутствие отопления в доме                                     2.24
T1 < min                                                        2.17
Температура в помещении общего пользования ниже нормативной     1.03
Сильная течь в системе отопления                                0.80
Течь в системе отопления                                        0.56
P2 <= 0                                                         0.53
Протечка труб в подъезде                                        0.38
P1 <= 0                                                         0.06
Name: count, dtype: float64

# Бинарный классификатор для каждого события отдельно
* на большом кол-ве событий - модель
* на малом могут быть правила

In [52]:
# 1. Данные для одного события
df_bin = df[df.event == "Температура в квартире ниже нормативной"]
df_bin["label"] = 1

/var/folders/3h/dhhfcbhd3pl2ss8czvd22c080000gn/T/ipykernel_13555/2957183093.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bin["label"] = 1


In [39]:
# 2. Добавить домов, у которых этого события не было
# В 11 приложении есть все дома из ВАО, у которых было отопление
no_event_df = districts_df[~districts_df.unom.isin(df_bin.unom.unique())]

In [51]:
# 3. Нужно подготовить те же признаки
# Пусть для начала это будут характеристики дома из 9 приложения
no_event_df = no_event_df.merge(building_info_df, how="left")
no_event_df["label"] = 0

In [60]:
# 4. Объединить в общий датафрейм
event_df = df_bin[["unom", "district", "material", "purpose", "class", "label"]]
event_df = event_df.drop_duplicates()
merge_df = pd.concat([event_df, no_event_df])

## Теперь можно попробовать построить классический бинарный классификатор

In [63]:
from sklearn.model_selection import train_test_split

In [68]:
merge_df[["label"]]

,label
0,1
6,1
13,1
18,1
46,1
...,...
981,0
982,0
983,0
984,0


In [69]:
X, y = merge_df[["unom", "district", "material", "purpose", "class"]], merge_df["label"]

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [78]:
from sklearn.ensemble import RandomForestClassifier

In [79]:
cls = RandomForestClassifier(n_estimators=100, random_state=42)

In [80]:
cls.fit(X_train, y_train)

ValueError: could not convert string to float: 'Преображенское'